# Algo trading project


In [ ]:
!pip3 install alpaca-trade-api

## Initialization

In [ ]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical import StockHistoricalDataClient# Create stock historical data client
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# Set API key and secret
api_key = 'your_api_key'
api_secret = 'your_api_secret'

# Initialize trading client for live trading (using paper trading mode)
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, StopLossOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from time import sleep

trading_client = TradingClient(api_key, api_secret, paper=True)

## Load historical data

In [ ]:
# Load historical data for backtesting
client = StockHistoricalDataClient(api_key, api_secret)
request_params = StockBarsRequest(
    symbol_or_symbols=["TSLA"],
    timeframe=TimeFrame.Day,
    start="2015-01-01 00:00:00",
    end="2023-06-01 00:00:00"
)
historical_data = client.get_stock_bars(request_params).df
print(historical_data.tail())

## Define strategy and backtesting methods

In [ ]:
def moving_average_strategy(data, short_window, long_window):
    data['short_mavg'] = data['close'].rolling(window=short_window, min_periods=1).mean()
    data['long_mavg'] = data['close'].rolling(window=long_window, min_periods=1).mean()

    data['signal'] = 0   # 1 for buy, -1 for sell, 0 for hold
    data['signal'][short_window:] = np.where(data['short_mavg'][short_window:] > data['long_mavg'][short_window:], 1, -1)

    data['strategy_return'] = data['signal'].shift(1) * data['close'].pct_change()

    return data

In [ ]:
def walk_forward_testing(data, short_window, long_window, train_period, test_period):
    results = []

    for start in range(0, len(data) - (train_period + test_period) + 1, train_period + test_period):
        train_data = data[start:start + train_period]
        test_data = data[start + train_period:start + train_period + test_period]

        strategy_train = moving_average_strategy(train_data.copy(), short_window, long_window)
        strategy_test = moving_average_strategy(test_data.copy(), short_window, long_window)

        results.append(strategy_test['strategy_return'].sum())  # cumulative return

    return results

In [ ]:
def monte_carlo_simulation(data, short_window, long_window, num_simulations=1000):
    simulation_results = []

    for _ in range(num_simulations):
        simulated_data = data.copy()
        simulated_data['close'] *= (1 + np.random.normal(0, 0.01, len(data)))  # Add random noise

        # Apply strategy to simulated data
        strategy_simulated = moving_average_strategy(simulated_data, short_window, long_window)
        strategy_returns = strategy_simulated['strategy_return'].dropna()

        # cumulative return and max drawdown
        cumulative_return = (1 + strategy_returns).prod() - 1
        max_drawdown = calculate_max_drawdown(strategy_returns)

        simulation_results.append((cumulative_return, max_drawdown))

    return simulation_results

In [ ]:
def calculate_sharpe_ratio(strategy_returns, risk_free_rate=0.02):
    excess_returns = strategy_returns.mean() - risk_free_rate / 252  # Adjust for daily returns
    return excess_returns / strategy_returns.std()

def calculate_max_drawdown(data):
    cumulative_returns = (1 + data['strategy_return']).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()
    return max_drawdown

In [ ]:
def walk_forward_testing(data, short_window_range, long_window_range, train_period, test_period):
    best_param = None
    best_score = -np.inf

    for start in range(0, len(data) - train_period - test_period + 1, test_period):
        train_data = data[start:start + train_period]
        test_data = data[start + train_period:start + train_period + test_period]

        best_sharpe = -np.inf
        best_long_window = None
        best_short_window = None

        # Optimize parameters on training data
        for short_window in short_window_range:
            for long_window in long_window_range:
                strategy_train = moving_average_strategy(train_data.copy(), short_window, long_window)
                strategy_returns = strategy_train['strategy_return'].dropna()

                sharpe_train = calculate_sharpe_ratio(strategy_returns)

                if sharpe_train > best_sharpe:
                    best_sharpe = sharpe_train
                    best_short_window = short_window
                    best_long_window = long_window

        # use the best parameters to evaluate on the test data
        strategy_test = moving_average_strategy(test_data.copy(), best_short_window, best_long_window)
        test_returns = strategy_test['strategy_return'].dropna()

        # performance metrics for the test data
        sharpe_test = calculate_sharpe_ratio(test_returns)
        max_drawdown_test = calculate_max_drawdown(test_returns)
        monte_carlo_results = monte_carlo_simulation(test_data, best_short_window, best_long_window)
        avg_cumulative_return = np.mean([result[0] for result in monte_carlo_results])
        avg_max_drawdown = np.mean([result[1] for result in monte_carlo_results])

        # *****CHOOSE METRIC {sharp_test, max_drawdown_test, avg_cumulative_return, avg_max_drawdown}*****
        score = sharpe_test

        # Update best parameters
        if score > best_score:
            best_score = score
            best_param = (best_short_window, best_long_window)

    return best_param

In [ ]:
train_period = 100
test_period = 20
short_window_range = range(10, 100, 10)
long_window_range = range(100, 300, 10)

best_parameters = walk_forward_testing(historical_data, short_window_range, long_window_range, train_period, test_period)

# Output the best parameters for future prediction
print(f'best short window = {best_parameters[0]}, best long window = {best_parameters[1]}')

## Backtest the strategy

In [ ]:
total_profit, trades = backtest_strategy(historical_data)
print(f"Total Profit from Backtesting: {total_profit}")
print(f"Individual Trades: {trades}")

# Live paper trading

In [ ]:
# Define parameters
symbol = "TSLA"
short_window = 50
long_window = 200
buy_qty = 10

# Initialize performance tracking
trade_log = []
total_profit = 0
total_trades = 0

def log_trade(order, trade_type, price):
    global total_profit, total_trades
    if trade_type == 'buy':
        trade_log.append({
            'type': 'buy',
            'price': price,
            'timestamp': pd.Timestamp.now()
        })
    elif trade_type == 'sell':
        last_trade = next((t for t in reversed(trade_log) if t['type'] == 'buy'), None)
        if last_trade:
            profit = price - last_trade['price']
            total_profit += profit
            total_trades += 1
            trade_log.append({
                'type': 'sell',
                'price': price,
                'timestamp': pd.Timestamp.now(),
                'profit': profit
            })
            print(f"Trade executed: Buy at {last_trade['price']}, Sell at {price}, Profit: {profit}")

def get_moving_averages():
    request_params = StockBarsRequest(
        symbol_or_symbols=[symbol],
        timeframe=TimeFrame.Day,
        start="2023-01-01 00:00:00"
    )
    historical_data = client.get_stock_bars(request_params).df
    historical_data['short_ma'] = historical_data['close'].rolling(window=short_window).mean()
    historical_data['long_ma'] = historical_data['close'].rolling(window=long_window).mean()
    return historical_data

In [ ]:
def execute_trade():
    historical_data_live = get_moving_averages()
    last_row = historical_data_live.iloc[-1]
    previous_row = historical_data_live.iloc[-2]

    if last_row['short_ma'] > last_row['long_ma'] and previous_row['short_ma'] <= previous_row['long_ma']:
        # Place a market buy order
        market_order_data = MarketOrderRequest(
            symbol=symbol,
            qty=buy_qty,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.GTC,
        )
        market_order = trading_client.submit_order(order_data=market_order_data)
        buy_price = float(market_order.filled_avg_price)
        print(f"Market buy order placed at {buy_price}")
        log_trade(market_order, 'buy', buy_price)

        # Set a stop-loss 5% below the buy price
        stop_loss_price = buy_price * 0.95
        stop_loss_order_data = StopLossOrderRequest(
            symbol=symbol,
            qty=buy_qty,
            side=OrderSide.SELL,
            stop_price=stop_loss_price,
            time_in_force=TimeInForce.GTC
        )
        trading_client.submit_order(order_data=stop_loss_order_data)
        print(f"Stop-loss order placed at {stop_loss_price}")

    elif last_row['short_ma'] < last_row['long_ma'] and previous_row['short_ma'] >= previous_row['long_ma']:
        # Place a market sell order (if in position)
        market_order_data = MarketOrderRequest(
            symbol=symbol,
            qty=buy_qty,
            side=OrderSide.SELL,
            time_in_force=TimeInForce.GTC,
        )
        market_order = trading_client.submit_order(order_data=market_order_data)
        sell_price = float(market_order.filled_avg_price)
        print(f"Market sell order placed at {sell_price}")
        log_trade(market_order, 'sell', sell_price)

In [ ]:
# Monitor live price updates and adjust positions
while True:
    try:
        execute_trade()
        # Print performance metrics
        print(f"Total Trades: {total_trades}, Total Profit: {total_profit}")
        sleep(60)  # Wait 1 minute before the next iteration
    except Exception as e:
        print(f"An error occurred: {e}")